In [1]:
#Research references:
#1) Dry/wet cough classification: https://link.springer.com/article/10.1007/s10439-013-0741-6
#2) Pneumonia classification: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6987276

In [2]:
import numpy as np
import os
import sox
#import pywt #wavelets
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
import matplotlib.pyplot as plt
import python_speech_features as spe_feats
import pandas as pd
from scipy.stats import kurtosis, skew, entropy
from scipy.signal import lfilter
import librosa
import math
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

## Settings

In [3]:
##DATASET

#folder of training data set
FOLDER_PATH = 'data/YT_set/wavs/'

#folder where normalized wavs are stored
NORM_FOLDER_PATH = 'data/YT_set/wavs/norm/'
norm_skip = False #skip normalization step (because it has been done previously)


##FEATUTRES

featExtr_skip = True

#Initialize data frame of features:

feats = pd.DataFrame([])

#tiny constant value
eps = sys.float_info.epsilon

#Features' settings:

fs_targ = 16000 # set all audios to this sampling frequency
n_channels_targ = 1

#framing
frame_len_s=0.025 #12 segments seemed adequeate in paper, since segments are no longer than 400ms (400ms/12=33.3ms)
frame_step_s=frame_len_s #according to paper: non-overlapping frames

frame_len = int(round(frame_len_s*fs_targ)) #in samples
frame_step = int(round(frame_step_s*fs_targ)) #in samples
win_func =np.hamming #at least for mfcc

#mfcc
cep_num= 13 #number of coefficients as in paper (https://link.springer.com/article/10.1007/s10439-013-0741-6)

#lp
lp_ord = int(round(2 + fs_targ/1000)) #standard rule of thumb for LP oder

#formants
nr_formants = 4 #as in paper, first 4 formants

## Functions

In [4]:
#Apply pre-emphasis (high-pass) filter
def apply_preEmph(x):
    x_filt = lfilter([1., -0.97], 1, x)
    return x_filt
        
#Obtain autocorrelation
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[int((result.size+1)/2):] #Note: other people use re.size/2:, but this does not work for me 
                                   # TODO: check consistency in other computers

#Compute zero-crossing rate
def get_zcr(x):
    zcr = (((x[:-1] * x[1:]) < 0).sum())/(len(x)-1)
    return zcr

#Compute log-energy
def get_logEnergy(x):
    logEnergy = np.log10( ( (np.power(x,2)).sum()/len(x) ) + eps)  
    return logEnergy

#Estimate fundamental frequency (F0)
def get_F0(x,fs):
    #autocorrelation-based method to extract F0
    xcorr_arr = autocorr(x)
    
    #looking for F0 in the frequency interval 50-500Hz, but we search in time domain
    min_ms = round(fs/500)
    max_ms = round(fs/50)
    
    xcorr_slot = xcorr_arr[max_ms+1:2*max_ms+1]
    xcorr_slot = xcorr_slot[min_ms:max_ms]
    t0 = np.argmax(xcorr_slot)
    F0 = fs/(min_ms+t0-1)
    return F0

#Estimate formants
def get_formants(x, lp_order, nr_formants):
    
    #compute lp coefficients
    a = librosa.lpc(x, lp_ord)
    

    #get roots from lp coefficients
    rts = np.roots(a)
    rts = [r for r in rts if np.imag(r) >= 0]

    #get angles
    angz = np.arctan2(np.imag(rts), np.real(rts))

    #get formant frequencies
    formants = sorted(angz * (fs_targ / (2 * math.pi)))
    
    return formants[0:nr_formants]

#Extract frequencies
def feature_extraction(x,fs,feats_df,lp_ord,ID,label):
#Extract features from signal x (identified as ID), and concatenate them to dataframe feats_df
#Features' reference: (see Appendix)
#[1]https://link.springer.com/article/10.1007/s10439-013-0741-6
#[2]https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?dsi_version=c5434db897ab74b192ca295a9eeca041&Expires=1585086202&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=c8k8DmG~KIxg0ToTO8rebm2MzHneCzJGkjSFRB7BYTEQ-MHXEr0ocHmISrldP3hFf9qmeiL11ezyefcNeRVeKIQ9PVjOl9pn7rXWcjA1o2voPn1VnDd8n7G2cT31apdj0LNMclhlXRPnCsGD66qDRqa3d-xaqqXhEqU73aw3ZgBgroO213MfJOqFhJxxXo2QEia0bSlDRTeX9KhSczFK-IFTPC6GwFL2L04por8pQRI3HF7E3f26O9zp9OhkwxSU9qfJah20WxZLA4PxREdv7JGoVBinR6T0mTcIaQi~B4IzYjSPSsTTADMNk5znVYIvSqgtMT~DY~qwlfq4SRdFjQ__
  
    
    #do features in a frame-basis
    x_frames = spe_feats.sigproc.framesig(x,frame_len,frame_step,win_func) #DOUBT: should I use window or not?
                                                                        #at least for formant estimation i should

    nr_frames = x_frames.shape[0]
    #print(nr_frames)
        
    #0)Wavelets #TODO
    
    #DOUBT: if log-energy feature is included, should I also include the first mfcc coefficient (c0) ?
    #1)mfcc
    mfcc_feat = spe_feats.mfcc(x,fs, winlen=frame_len_s,winstep=frame_step_s, numcep=cep_num,winfunc=win_func)
    
    #deltas to capture 
    mfcc_delta_feat = spe_feats.delta(mfcc_feat,1) #mfcc_delta_feat = np.subtract(mfcc_feat[:-1], mfcc_feat[1:]) #same
    mfcc_deltadelta_feat = spe_feats.delta(mfcc_delta_feat,1)          
    
    #2)zero-crossing rate
    zcr_feat = np.apply_along_axis(get_zcr, 1, x_frames)
    
    #3)Formant frequencies
    #using LP-coeffcs-based method
    #formant_feat = np.apply_along_axis(get_formants, 1, x_frames, lp_ord, nr_formants)
    
    #Note: for the moment, it seems some frames are ill-conditioned for lp computing,
    #current solution - we skip those and fill with NaN values
    formants_feat= np.empty((nr_frames,4))
    formants_feat[:] = np.nan
    
    for i_frame in range(0,nr_frames):
        try: 
            formants_feat[i_frame] = get_formants(x_frames[i_frame], lp_ord, nr_formants)
        except:
            pass
    
    #4)Log-energy
    logEnergy_feat =  np.apply_along_axis(get_logEnergy, 1, x_frames)
    
    #5)Pitch (F0)
    F0_feat =  np.apply_along_axis(get_F0, 1, x_frames,fs)
    
    #TODO: compute also F0 with pysptk (a python wrapper for SPTK library), it probably gives better results
    #https://github.com/r9y9/pysptk/blob/master
    
    #6)Kurtosis
    kurt_feat =  np.apply_along_axis(kurtosis, 1, x_frames)
    
    #7)Bispectrum Score (BGS)
    #TODO: see PhD thesis for more info on this feature
    
    #8)Non-Gaussianity Score (NGS)
    #TODO: see PhD thesis for more info on this feature
   
    #9) Adding skewness as measure of non-gaussianity (not in paper)
    skew_feat =  np.apply_along_axis(skew, 1, x_frames)
    
    #DOUBT: 10) Shannon entropy GETTING -inf in all cases, WHY??? Don't include until fixed
    #entropy_feat = entropy(x)
    #Maybe compute directly to check
    
    mfcc_cols = ['mfcc_%s' % s for s in range(0,cep_num)]
    mfcc_delta_cols = ['mfcc_d%s' % s for s in range(0,cep_num)]
    mfcc_deltadelta_cols = ['mfcc_dd%s' % s for s in range(0,cep_num)]
    formants_cols = ['F%s' % s for s in range(1,nr_formants+1)]
          
    feats_segment = pd.concat([pd.DataFrame({'Id': ID, 'kurt': kurt_feat, 'logEnergy': logEnergy_feat,
                                                 'zcr': zcr_feat, 'F0': F0_feat,
                                                 'skewness': skew_feat, 'label': label}),
                               pd.DataFrame(mfcc_feat,columns=mfcc_cols), 
                            pd.DataFrame(formants_feat,columns=formants_cols)],axis=1)
    
    print(nr_frames)
    feats_df = feats_df.append(feats_segment,ignore_index=True, sort=False)
    
    return feats_df


# MAIN

## Reading recordings + feature extraction

In [5]:
#Read wav data set, apply pre-processing and extract features

if featExtr_skip is False:

    #only list files in FOLDER_PATH directory
    wav_files = [f for f in os.listdir(FOLDER_PATH) if os.path.isfile(os.path.join(FOLDER_PATH, f))]
    for file_name in wav_files:
    
        fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
        #full path file name
        full_fname = FOLDER_PATH+file_name
        #print(full_fname)
    
        #name for normalization
        norm_fname = NORM_FOLDER_PATH + os.path.splitext(file_name)[0] + '_NORM.wav'
    
        if norm_skip is False: 
        ## Normalization
        
            #level to same dB
            tfm = sox.Transformer()
            tfm.gain(gain_db=0.0, normalize=False, limiter=False, balance=None)
            #downsample to 16kHz and 1 channel
            tfm.convert(samplerate=fs_targ, n_channels=n_channels_targ, bitdepth=None) 
            #tfm.norm(db_level=0.0)
    
            # create the output normalized audio
        
            print(norm_fname)
            tfm.build(full_fname, norm_fname)
            tfm.effects_log
    
        # load normalized audio
        s = AudioSegment.from_wav(norm_fname)
        #sampling rate:
        info = mediainfo(norm_fname)
        fs = float(info['sample_rate'])
    
        #get ID of recording
        ID = fname_noExt.split('-')[-2] #for the current type of naming
        print(file_name)
        print(ID)
    
        #get label
        label = fname_noExt.split('-')[-1] #for the current type of naming
        print(label)
    
        ## Segmentation of cough streams (silence-based)
        #min_silence_len in ms, silence_thresh in dB
        s_segments = split_on_silence (s, min_silence_len = 600, silence_thresh = -30)
        ## TODO: set more accurate thresholds, or find other way to split (variance-based?)
    
        #convert s_segments to numpy array format
        AudioSegment2numpy_arr = lambda x: np.asarray(x.get_array_of_samples())
        s_segments_np = list(map(AudioSegment2numpy_arr, s_segments))
    
        print('High-pass filtering...')
        #pre-emphasis filtering to each segment
        preEmph_filtering = lambda x: apply_preEmph(x)
        s_segments_filt = list(map(preEmph_filtering, s_segments_np))
    
        print('Computing features...')
        #Feature extraction for each segment
    
        #(lambda function doesn't work )
        #feat_extr_step = lambda x, fs, feats_df, lp_ord, ID: feature_extraction(x,fs,feats_df,lp_ord,ID)
        #feats = feat_extr_step(s_segments_filt,fs,feats,lp_ord,ID)
        for idx, seg_i in enumerate(s_segments_filt):
            print('\tSegment %d' % idx)
            feats = feature_extraction(seg_i,fs,feats,lp_ord,ID,label)
    
       

## Load  (or store) features 

In [6]:

feats_fname = 'feats_df.pkl'

if featExtr_skip is False:
    #Store feature df
    feats.to_pickle(feats_fname)
else:
    #Load feature df
    feats = pd.read_pickle(feats_fname)

## Pre-processing of features

In [7]:
#1.Check which columns have NaNs values

#feats2 = feats.copy()

#sum(feats.isna().any())
#feats.columns[feats.isna().any()].tolist() --> We get just the ones we have inserted in formants
feats2 = feats.interpolate(method ='cubic')
#feats2 = feats.dropna(axis=0).copy()
#feats2.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)

#feats2.columns[feats2.isna().any()].tolist()
#feats2.describe()


In [8]:
sum(feats2.isna().any())

0

In [9]:
feats2['cum_IDidx'] = feats2.groupby('Id').cumcount()

In [10]:
feats2.columns

Index(['Id', 'kurt', 'logEnergy', 'zcr', 'F0', 'skewness', 'label', 'mfcc_0',
       'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7',
       'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'F1', 'F2', 'F3',
       'F4', 'cum_IDidx'],
      dtype='object')

In [11]:
def get_subidx(cum_Idx,batch_size):
    #batch needs to be an integer (or float like 3.0)
    return int(1.0*cum_Idx/batch_size)

feats2['subIdx'] = feats2.apply(lambda x: get_subidx(x['cum_IDidx'], 10), axis=1)
feats2 = feats2.drop(['cum_IDidx'],axis=1)

In [12]:
feats2

,Id,kurt,logEnergy,zcr,F0,skewness,label,mfcc_0,mfcc_1,mfcc_2,...,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,F1,F2,F3,F4,subIdx
0,Q,2.130898,5.535231,0.493734,500.000000,-0.073570,Dry,18.726310,-49.164042,-21.943514,...,-6.405460,-23.258862,-30.374339,21.403037,24.980936,0.000000,925.103724,1840.988979,2514.599705,0
1,Q,1.545643,5.650084,0.481203,355.555556,-0.023850,Dry,18.853876,-46.084848,-23.129777,...,-21.431062,2.649611,-2.911352,23.824741,6.022679,984.918270,1336.004572,2393.647660,3457.490333,0
2,Q,3.455492,5.693125,0.398496,340.425532,0.060297,Dry,18.881794,-46.953603,-29.814847,...,-33.612875,0.582441,-8.900234,18.914764,21.723671,1082.497991,1521.894130,2303.224957,3334.387099,0
3,Q,2.705560,5.950192,0.373434,390.243902,-0.028460,Dry,19.155061,-45.559214,-34.543965,...,-55.256140,1.971662,4.050639,26.670061,15.960065,1137.112141,1159.306893,2339.993180,3205.991440,0
4,Q,1.262319,5.939354,0.270677,400.000000,0.261261,Dry,18.881250,-47.445340,-34.115887,...,-40.710783,-5.962290,11.440152,25.719434,-5.918450,1157.686238,1232.201808,2319.610594,3314.860067,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26066,1UDFq2InljM,3.047776,6.269337,0.182957,470.588235,-1.157113,Dry,18.547338,-27.690078,-19.460858,...,-24.957201,-35.001481,5.110327,-22.201085,-5.170684,0.000000,485.811290,1127.858286,2362.825897,36
26067,1UDFq2InljM,2.946299,4.816438,0.290727,444.444444,-0.049535,Dry,16.141393,-29.294489,-13.170555,...,-44.159174,-19.803384,16.631558,6.202915,-5.215586,403.598514,1156.610226,2403.494580,3549.297585,36
26068,1UDFq2InljM,2.435713,4.248565,0.310777,516.129032,0.039825,Dry,14.692141,-27.883082,-13.303023,...,-27.726053,-0.864033,4.824035,-10.168872,-22.813406,0.000000,565.937275,1223.541129,2561.363375,36
26069,1UDFq2InljM,2.741493,3.926815,0.421053,421.052632,0.242132,Dry,14.522135,-32.537145,-11.114888,...,-34.327166,-8.661193,-0.349616,-9.037070,-4.645845,0.000000,442.061606,1276.099474,2520.313825,36


In [13]:
mean_feats = feats2.groupby(['Id','subIdx']).aggregate('mean').reset_index()
std_feats = feats2.groupby(['Id','subIdx']).agg(lambda x: x.std(ddof=0)).reset_index() #ddof=0 to compute population std (rather than sample std)
keep_same = {'Id', 'subIdx'}
mean_feats.columns = ['{}{}'.format(c, '' if c in keep_same else '_m') for c in mean_feats.columns]
std_feats.columns = ['{}{}'.format(c, '' if c in keep_same else '_std') for c in std_feats.columns]

In [14]:
sum(std_feats.isna().any())

0

In [15]:
mean_std_feats = pd.merge(mean_feats, std_feats, on=['Id','subIdx'], how='outer')

In [16]:
#Make dictionary and add label column using it 
feats_unique = feats.drop_duplicates(subset=['Id'])
label_dict = dict(zip(feats_unique.Id, feats_unique.label))
mean_std_feats['label'] = mean_std_feats["Id"].map(label_dict)
#mean_std_feats[['Id','label']].head(50)

In [17]:
mean_std_feats.columns

Index(['Id', 'subIdx', 'kurt_m', 'logEnergy_m', 'zcr_m', 'F0_m', 'skewness_m',
       'mfcc_0_m', 'mfcc_1_m', 'mfcc_2_m', 'mfcc_3_m', 'mfcc_4_m', 'mfcc_5_m',
       'mfcc_6_m', 'mfcc_7_m', 'mfcc_8_m', 'mfcc_9_m', 'mfcc_10_m',
       'mfcc_11_m', 'mfcc_12_m', 'F1_m', 'F2_m', 'F3_m', 'F4_m', 'F0_std',
       'F1_std', 'F2_std', 'F3_std', 'F4_std', 'kurt_std', 'logEnergy_std',
       'mfcc_0_std', 'mfcc_1_std', 'mfcc_10_std', 'mfcc_11_std', 'mfcc_12_std',
       'mfcc_2_std', 'mfcc_3_std', 'mfcc_4_std', 'mfcc_5_std', 'mfcc_6_std',
       'mfcc_7_std', 'mfcc_8_std', 'mfcc_9_std', 'skewness_std', 'zcr_std',
       'label'],
      dtype='object')

In [18]:
mean_std_feats.describe()

,subIdx,kurt_m,logEnergy_m,zcr_m,F0_m,skewness_m,mfcc_0_m,mfcc_1_m,mfcc_2_m,mfcc_3_m,...,mfcc_2_std,mfcc_3_std,mfcc_4_std,mfcc_5_std,mfcc_6_std,mfcc_7_std,mfcc_8_std,mfcc_9_std,skewness_std,zcr_std
count,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,...,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000,2626.000000
mean,50.594440,3.014324,5.057183,0.442242,436.507056,0.025299,17.293175,-41.088863,-16.290866,-18.048724,...,6.958737,7.778120,9.352091,10.046279,10.093717,11.104148,10.564540,11.042874,0.290360,0.085928
std,44.225823,2.114979,1.139403,0.119214,23.649240,0.158403,2.894618,9.581659,8.964917,10.056405,...,2.793573,2.924535,3.479074,3.634197,3.469164,4.162364,3.718430,3.923377,0.233485,0.044152
min,0.000000,0.795110,-7.143552,0.027569,323.309617,-1.424550,-13.401690,-64.068463,-51.915135,-69.045057,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,2.187196,4.484446,0.354386,422.416762,-0.050559,15.780162,-48.167792,-21.960196,-24.739993,...,4.942332,5.688186,6.791556,7.426078,7.590721,8.118966,7.884444,8.270781,0.186204,0.050497
50%,40.000000,2.547719,5.179256,0.454010,438.487616,0.014647,17.546049,-42.283133,-16.081892,-17.532042,...,6.599567,7.359867,8.855415,9.701592,9.688289,10.498203,10.046575,10.510245,0.238337,0.078891
75%,69.000000,3.217146,5.814389,0.535526,452.914297,0.085358,19.242631,-34.774276,-10.363300,-11.377825,...,8.568864,9.410180,11.445713,12.206047,12.029965,13.356861,12.672039,13.264684,0.321304,0.114440
max,229.000000,57.708003,8.081837,0.738346,516.129032,3.058542,24.863263,7.360785,16.392344,23.993110,...,20.589080,24.924838,26.185592,25.159065,24.553697,34.997448,30.671556,33.594764,6.233596,0.278195


In [19]:
mean_std_feats.columns[mean_std_feats.isna().any()].tolist() 

[]

In [20]:
mean_std_feats['Id']

0       1UDFq2InljM
1       1UDFq2InljM
2       1UDFq2InljM
3       1UDFq2InljM
4       1UDFq2InljM
           ...     
2621    zjd4HrJbc8o
2622    zjd4HrJbc8o
2623    zjd4HrJbc8o
2624    zjd4HrJbc8o
2625    zjd4HrJbc8o
Name: Id, Length: 2626, dtype: object

In [21]:
sum(mean_std_feats.isna().any())

0

In [22]:
#mean_std_feats = mean_std_feats.interpolate(method ='cubic')
#mean_std_feats.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True) #--> Doesn't work!? Still get NaN error

In [23]:
sum(mean_std_feats.isna().any())

0

In [24]:
mean_std_feats['Id']

0       1UDFq2InljM
1       1UDFq2InljM
2       1UDFq2InljM
3       1UDFq2InljM
4       1UDFq2InljM
           ...     
2621    zjd4HrJbc8o
2622    zjd4HrJbc8o
2623    zjd4HrJbc8o
2624    zjd4HrJbc8o
2625    zjd4HrJbc8o
Name: Id, Length: 2626, dtype: object

In [25]:
#2. Get feature set, labels, and recording IDs
X_train = mean_std_feats.drop(['label','Id','subIdx'], 1).copy()
y_train =  mean_std_feats['label'].copy()

ID_train = mean_std_feats['Id']
ID_list = ID_train.drop_duplicates()

#ID_train.size
ID_list.size

36

In [26]:
#3. Normalization in case some model requires it

scaler = StandardScaler()
scaler.fit(X_train)

#use same scaler for both, based on X_train data
X_trainNorm = scaler.transform(X_train.values)

In [27]:
sum(X_train.isna().any())

0

## Model training

### Train-test split (k-fold)

In [28]:
k = ID_list.values.size #number of folds

group_kfold = GroupKFold(n_splits=k)
group_kfold.get_n_splits(X_trainNorm, y_train, ID_train)

36

### Logistic regression

In [65]:
#Do cross-validation
pred_probs = pd.DataFrame([])

idx_acc = 0
for train_index, test_index in group_kfold.split(X_trainNorm,y_train,ID_train):
    X_train1, X_test1 = X_trainNorm[train_index], X_trainNorm[test_index]
    y_train1, y_test1 = y_train[train_index], y_train[test_index]
    
    #logReg = LogisticRegression()
    
    #loss=log --> logistic regression
    logReg = SGDClassifier(loss='log', penalty='l2')
    logReg.fit(X_train1, y_train1)
    y_hat_prob = logReg.predict_proba(X_test1)
    classes =logReg.classes_
    pred_probs = pred_probs.append(pd.DataFrame({'ID': ID_train[test_index], str(classes[0]): y_hat_prob[:,0], str(classes[1]): y_hat_prob[:,1]}),ignore_index=True, sort=False)    

In [54]:
#mean_feats = feats2.groupby(['Id','subIdx']).aggregate('mean').reset_index()
mean_pred_probs = pred_probs.groupby('ID').aggregate('mean').reset_index()

In [43]:
def predict_class(prob_dry,prob_wet):
    if prob_dry > prob_wet :
        return 'Dry'
    else:
        return 'Wet'
    
mean_pred_probs['pred_class'] = mean_pred_probs.apply(lambda x: predict_class(x['Dry'], x['Wet']), axis=1)

In [44]:
#add actual classes
mean_pred_probs['label'] = mean_pred_probs["ID"].map(label_dict)

In [45]:
mean_pred_probs

,ID,Dry,Wet,pred_class,label
0,1UDFq2InljM,0.558686,0.441314,Dry,Dry
1,4k0ziD0j5BI,0.516737,0.483263,Dry,Wet
2,5905FxXz9dI,0.711460,0.288540,Dry,Wet
3,6LK6yHtIung,0.693342,0.306658,Dry,Dry
4,7Ez5Wc_esBg,0.784853,0.215147,Dry,Dry
5,A5s2ZgwQ1VM,0.717208,0.282792,Dry,Dry
6,AQOeIVbhFm4,0.739964,0.260036,Dry,Dry
7,CTSLdNxN1cc,0.474885,0.525115,Wet,Wet
8,CsDXlt7Ei1c,0.233362,0.766638,Wet,Wet
9,DYfjPnty2Ho,0.462289,0.537711,Wet,Wet


## Evaluation

In [46]:
#Accuracy
acc = accuracy_score(mean_pred_probs['label'], mean_pred_probs['pred_class'])
print(acc)

0.6944444444444444


In [47]:
#TODO: Check if following measures are computed OK

In [48]:
#Precision
prec = precision_score(mean_pred_probs['label'], mean_pred_probs['pred_class'],average="macro")
print(prec)

0.6875


In [49]:
#F1-score
f1 = f1_score(mean_pred_probs['label'], mean_pred_probs['pred_class'],average="macro")
print(f1)

0.674074074074074


In [50]:
#recall
recall = recall_score(mean_pred_probs['label'], mean_pred_probs['pred_class'],average="macro")
print(recall)

0.6714285714285715


In [51]:
#TODO: check confusion matrix?, to see where there was the most problems --> I can see directly in the df

In [52]:
#Cough sound
#Breathing rate
#Breathing rhytm (consistence smoothness)
#Cough rate
#Panic level
#Hoarseness